In [7]:
mainGitHub='https://raw.github.com/meade68/DACSS_690D_Week09_HW2/raw/refs/heads/main/'
BG=mainGitHub+"DATA/tl_2025_25_tabblock20.zip"

import geopandas as gpd
URL=mainGitHub+BG


census_BG= gpd.read_file(f"zip://{URL}")

DataSourceError: '/vsizip/https://raw.github.com/meade68/DACSS_690D_Week09_HW2/raw/refs/heads/main/https://raw.github.com/meade68/DACSS_690D_Week09_HW2/raw/refs/heads/main/DATA/tl_2025_25_tabblock20.zip' does not exist in the file system, and is not recognized as a supported dataset name.